# Tutorial for auto differentiation package

This package created by @Faris-ML

### How to differentiate automaticlly?

The derivative is taken by going throuth the following steps:

1- Define the equation by the package operator

2- Create a graph

3- compute the forward pass and backward pass

### Methodology:

The way to be able to take derivative is to convert the equation to graph.

example:

let say that we want to evaluate this equation

y = (x^2)+6

after converting the equation to graph will be like this:

![image](images/graph.png)

and by the chain rule we compute the gradients by taking the backward pass like the graph below :

![image](images/d_graph.png)

## lets get started on implementation

In [19]:
import Graph as g
import Node as n

Define variables and constant

In [20]:
x = n.Variable(3,name='x')
cons_1 = n.Constant(2,name='cons_1')
cons_2 = n.Constant(6,name='cons_1')

In [31]:
y = n.add(n.power(x,cons_1),cons_2)
for i in range(500):
    y = n.add(n.power(x,cons_1),y)

Create a graph

In [32]:
graph = g.Graph(y)

In [33]:
import time

Compute forward pass and backward pass

In [35]:
s = time.time()
forward = graph.forward()
print('the forward pass : ',forward)
backward = graph.backward(just_Variables=True)
print('the backward pass : ',backward)
print('as you see the backward pass return a dictionary that the key is variable name and the value is the variable object')
print('we can get the backward value or the gradients by using a grad attribute in the object : ')
print('the derivative with respect to x is : ',backward['x'].grad)
e = time.time()
print((e-s)*1000)

the forward pass :  4515
the backward pass :  {'x': <Node.Variable object at 0x00000273CE50ED80>}
as you see the backward pass return a dictionary that the key is variable name and the value is the variable object
we can get the backward value or the gradients by using a grad attribute in the object : 
the derivative with respect to x is :  3006
3.2634735107421875
